# The Agents of FloAI

This notebook shows different kinds of agents that can be built using flo-ai. These agents have their own specialities, and this notebook tries to explain them.

Here are the major types of agents:

1. Agentic Agents (`kind: agentic`): Any agent created in flo-ai by default tries to be an agentic agent. This agent always needs a tool, meaning an agent without a tool if specified as agentic agent whill throw an exception

2. LLM Agents (`kind: llm`): These agents are simply an LLM which can answer any questions asked to it. The agents dont except tools. If tool is passed to an agent of type llm, they are ignored.

3. Tool LLM (`kind: tool`): These agents are just tools or functions that can be executed on the current state. Within the tool will be given the current state as the input, meaning the history of what happened in the flo until now

In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from flo_ai.common.flo_logger import get_logger
from flo_ai.common.flo_callback_handler import FloCallbackHandler
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

## Setup
Create the Flo session, setup tools

In [2]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')

# Create custom logger and callback handler
custom_logger_new = get_logger("CustomFloSession", "DEBUG")
custom_callback = FloCallbackHandler("CustomFloCallback", "DEBUG")

session = FloSession(
    llm, 
    log_level="DEBUG",
    custom_logger=custom_logger_new,
    custom_callback_handler=custom_callback
)


session.register_tool(
    name="InternetSearchTool", 
    tool=TavilySearchResults()
)

2024-09-17 13:57:40,859 - CustomFloSession - INFO - New FloSession created with ID: 3ec65857-96ef-40c0-8323-6f3e5cf8dbda
2024-09-17 13:57:40,861 - CustomFloSession - INFO - Tool 'InternetSearchTool' registered for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda


## Agentic Agent (agentic)

Here we are gonna create a simple weather assitant flo agent that can check the whether by an internet searching tool.

As you can see the kind is `agentic`

In [3]:
simple_weather_checking_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: weather-assistant
agent:
    name: Weather Assistant
    kind: agentic
    job: >
      Given the city name you are capable of answering the latest whether this time of the year by searching the internet
    tools:
      - name: InternetSearchTool
"""

In [4]:
flo = Flo.build(session, simple_weather_checking_agent)

flo.invoke("Whats the whether in california")

2024-09-17 13:57:47,876 - Flo.build - INFO - Building Flo instance from YAML
2024-09-17 13:57:48,292 - CustomFloSession - INFO - Flo instance created for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda
2024-09-17 13:57:48,293 - CustomFloSession - INFO - Invoking query for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda: Whats the whether in california




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726561652, 'localtime': '2024-09-17 01:27'}, 'current': {'last_updated_epoch': 1726560900, 'last_updated': '2024-09-17 01:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14.8, 'wind_degree': 255, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.9, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 51, 'cloud': 0, 'feelslike_c': 12.0, 'feelslike_f': 53.6, 'windchill_c': 5.7, 'windchill_f': 42.3, 'heatindex_c': 8.2, 'heatindex_f': 46.7, 'dewpoint_c': 6.1, 'dewpoint_f': 43.0, 'vis_

{'messages': [HumanMessage(content='Whats the whether in california')],
 'output': 'The weather in California during October 2023 has varied across different regions. Here are some highlights:\n\n1. **Current Weather**: As of now, in California City, the temperature is approximately 13.3°C (55.9°F) with clear skies. The wind is blowing from the west-southwest at about 9.2 mph, and the humidity is around 51%.\n\n2. **Los Angeles**: The weather reports indicate that Los Angeles experienced a high of 92°F on October 5, with humidity reaching 100% on October 7.\n\n3. **General Forecast**: Throughout October, temperatures have ranged from highs in the upper 70s to low 90s°F, with lows typically in the 50s°F. \n\nFor more detailed and specific forecasts, you can check resources like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).'}

In [5]:
# with logging

from flo_ai.common.flo_callback_handler import FloCallbackHandler

flo = Flo.build(session, simple_weather_checking_agent)

handler = FloCallbackHandler(log_level="INFO")

config = {
    'callbacks' : [handler]
}

query="Whats the whether in california"

flo.invoke(query=query, config=config)

2024-09-17 13:58:00,020 - Flo.build - INFO - Building Flo instance from YAML
2024-09-17 13:58:00,036 - CustomFloSession - INFO - Flo instance created for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda
2024-09-17 13:58:00,039 - CustomFloSession - INFO - Invoking query for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda: Whats the whether in california
2024-09-17 13:58:00,046 - FloCallback - INFO - onChainStart: {'messages': [HumanMessage(content='Whats the whether in california')]}
2024-09-17 13:58:00,073 - FloCallback - INFO - onChainStart: {'input': ''}
2024-09-17 13:58:00,079 - FloCallback - INFO - onChainStart: {'input': ''}
2024-09-17 13:58:00,093 - FloCallback - INFO - onChainStart: {'input': ''}
2024-09-17 13:58:00,106 - FloCallback - INFO - onChainStart: {'input': ''}
2024-09-17 13:58:00,108 - FloCallback - INFO - onChainEnd: []
2024-09-17 13:58:00,110 - FloCallback - INFO - onChainEnd: {'agent_scratchpad': []}
2024-09-17 13:58:00,112 - FloCallback - INFO - onChainEnd: {'messages': [



> Entering new AgentExecutor chain...


2024-09-17 13:58:01,034 - FloCallback - INFO - onLLMEnd: [[ChatGenerationChunk(generation_info={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857'}, message=AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ynjcmGUHsAvTrXuWTuhrAP9X', 'function': {'arguments': '{"query":"California weather October 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857'}, id='run-91b02103-c7e0-4c25-a978-ea86019e7a44', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'California weather October 2023'}, 'id': 'call_ynjcmGUHsAvTrXuWTuhrAP9X', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '{"query":"California weather October 2023"}', 'id': 'call_ynjcmGUHsAvTrXuWTuhrAP9X', 'index': 0, 'type': 'tool_call_chunk'}]))]


Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`




2024-09-17 13:58:05,355 - FloCallback - INFO - onToolEnd: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726561652, 'localtime': '2024-09-17 01:27'}, 'current': {'last_updated_epoch': 1726560900, 'last_updated': '2024-09-17 01:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14.8, 'wind_degree': 255, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.9, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 51, 'cloud': 0, 'feelslike_c': 12.0, 'feelslike_f': 53.6, 'windchill_c': 5.7, 'windchill_f': 42.3, 'heatindex_c': 8.2, 'heatindex_f': 46.7, 'dewpoint_c': 6.1, 'dewpoint_f': 43.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 16.7, 'gust_kph': 26.8}}

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726561652, 'localtime': '2024-09-17 01:27'}, 'current': {'last_updated_epoch': 1726560900, 'last_updated': '2024-09-17 01:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14.8, 'wind_degree': 255, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.9, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 51, 'cloud': 0, 'feelslike_c': 12.0, 'feelslike_f': 53.6, 'windchill_c': 5.7, 'windchill_f': 42.3, 'heatindex_c': 8.2, 'heatindex_f': 46.7, 'dewpoint_c': 6.1, 'dewpoint_f': 43.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 16.7, 'gust_kph': 26.8}}"}, {'url': 'https://www.timeanddate.com/weather/usa/los-a

2024-09-17 13:58:08,478 - FloCallback - INFO - onLLMEnd: [[ChatGenerationChunk(text='The weather in California during October 2023 has varied across different regions. Here are some highlights:\n\n1. **Current Weather**: As of now, in California City, the temperature is approximately 13.3°C (55.9°F) with clear skies. The wind is blowing from the west-southwest at about 9.2 mph, and the humidity is around 51%.\n\n2. **Los Angeles**: The weather reports indicate that Los Angeles experienced a high of 92°F on October 5, with humidity reaching 100% on October 7.\n\n3. **General Forecast**: Throughout October, temperatures have ranged from highs in the upper 70s to low 90s°F, with lows typically in the 50s°F. \n\nFor more detailed and specific forecasts, you can check resources like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).', generation_info={'finish_reason': 'stop', 'model_name': 'gpt-4o-mi

The weather in California during October 2023 has varied across different regions. Here are some highlights:

1. **Current Weather**: As of now, in California City, the temperature is approximately 13.3°C (55.9°F) with clear skies. The wind is blowing from the west-southwest at about 9.2 mph, and the humidity is around 51%.

2. **Los Angeles**: The weather reports indicate that Los Angeles experienced a high of 92°F on October 5, with humidity reaching 100% on October 7.

3. **General Forecast**: Throughout October, temperatures have ranged from highs in the upper 70s to low 90s°F, with lows typically in the 50s°F. 

For more detailed and specific forecasts, you can check resources like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).

> Finished chain.


{'messages': [HumanMessage(content='Whats the whether in california')],
 'output': 'The weather in California during October 2023 has varied across different regions. Here are some highlights:\n\n1. **Current Weather**: As of now, in California City, the temperature is approximately 13.3°C (55.9°F) with clear skies. The wind is blowing from the west-southwest at about 9.2 mph, and the humidity is around 51%.\n\n2. **Los Angeles**: The weather reports indicate that Los Angeles experienced a high of 92°F on October 5, with humidity reaching 100% on October 7.\n\n3. **General Forecast**: Throughout October, temperatures have ranged from highs in the upper 70s to low 90s°F, with lows typically in the 50s°F. \n\nFor more detailed and specific forecasts, you can check resources like [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023).'}

## LLM Agent (agentic)

Here we are gonna create a simple llm math assitant flo agent that can check answer any math question

As you can see the kind is `llm`

In [6]:
simple_llm_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: Ask llm anything
    kind: llm
    job: >
      You are a high school maths teacher. Answer any questions the students ask 
"""

In [7]:
flo = Flo.build(session, simple_llm_agent)

flo.invoke("What is pythagorus theorum")
# print(flo.invoke("What is pythagorus theorum"))

2024-09-17 14:00:37,552 - Flo.build - INFO - Building Flo instance from YAML


2024-09-17 14:00:37,585 - CustomFloSession - INFO - Flo instance created for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda
2024-09-17 14:00:37,588 - CustomFloSession - INFO - Invoking query for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda: What is pythagorus theorum


"Pythagoras' Theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. \n\nThe theorem can be expressed with the formula:\n\n\\[ c^2 = a^2 + b^2 \\]\n\nwhere:\n- \\( c \\) is the length of the hypotenuse,\n- \\( a \\) and \\( b \\) are the lengths of the other two sides.\n\nThis theorem is very useful for calculating distances and for solving various problems in geometry. If you have any specific questions or examples you'd like to go through, feel free to ask!"

#

In [8]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

class PrintStateTool(BaseTool):
    name = "printStateTool"
    description = "Just print the state"

    def _run(
        self, **kwargs
    ) -> str:
        print(kwargs)
        return "Tool call success"
    
session.register_tool(
    name="printStateTool", 
    tool=PrintStateTool()
)
    
simple_tool_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: tool-to-print-state
    kind: tool
    tools:
        - name: printStateTool
"""

flo = Flo.build(session, simple_tool_agent)

flo.invoke("Print what I am saying")

2024-09-17 14:00:42,750 - CustomFloSession - INFO - Tool 'printStateTool' registered for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda
2024-09-17 14:00:42,754 - Flo.build - INFO - Building Flo instance from YAML
2024-09-17 14:00:42,763 - CustomFloSession - INFO - Flo instance created for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda
2024-09-17 14:00:42,768 - CustomFloSession - INFO - Invoking query for session 3ec65857-96ef-40c0-8323-6f3e5cf8dbda: Print what I am saying


{'messages': [HumanMessage(content='Print what I am saying')]}


'Tool call success'